In [ ]:
#| default_exp datastructure.odds

In [ ]:
#| include: false
from IPython.core.debugger import set_trace

# Odds

In this document we will define a data structure for Odds and We are mainly interested by 3 types of markets:

   - 1x2
   - asian-handicap
   - over/under


In [ ]:
#| export

import datetime
import mongoengine
import re
import pandas as pd
import warnings
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from betting_env.config.localconfig import CONFIG
from betting_env.config.mongo import mongo_init
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


MARKET_TYPES = ("asian", "total", "1x2")


class MarketOdds(mongoengine.Document):
    """
    Store the odds values

    Attributes
    ----------
    time_received: datetime
        when the odds were recived
    odds1,odds2: float
        odds for home and away
    oddsX: float, optional
        odds for draw when defined
    gameDate: datetime
        game date time
    season_id: int
        seasonId
    competition_Id:str
        competition-id
    home_team_id, away_team_id: str
        home and away team ids
    game_id: str
        game-identifier
    line_id: float
        when defined, the asian or over/under line
    bookmaker_id: int
        bookmaker-id
    bookmaker_name: str
        bookmaker-name
    market_type: str
        market-type
    overround: flaot
        bookmaker margin or overround
    """

    ## game-info
    game_id = mongoengine.StringField(db_field="gameId", required=True)
    game_opta_id = mongoengine.IntField(db_field="game_optaId", required=False)
    game_date = mongoengine.DateTimeField(db_field="gameDate", required=True)
    competition_id = mongoengine.StringField(db_field="competitionId", required=True)
    competition_opta_id = mongoengine.IntField(
        db_field="competition_optaId", required=False
    )
    season_id = mongoengine.IntField(db_field="seasonId", required=True)
    home_team_id = mongoengine.StringField(db_field="homeTeamId", required=True)
    away_team_id = mongoengine.StringField(db_field="awayTeamId", required=True)

    ## bookmaker info
    bookmaker_id = mongoengine.IntField(db_field="bookmakerId", required=True)
    bookmaker_name = mongoengine.IntField(db_field="bookmakerName", required=True)
    source = mongoengine.StringField(required=False)

    ## market info
    market_type = mongoengine.StringField(
        choices=MARKET_TYPES, db_field="marketType", required=True
    )
    line_id = mongoengine.FloatField(db_field="lineId", required=False)

    ## odds
    received_at = mongoengine.DateTimeField(db_field="timeReceived", required=True)
    odds1 = mongoengine.FloatField(required=True, min_value=1)
    odds2 = mongoengine.FloatField(required=True, min_value=1)
    oddsX = mongoengine.FloatField(required=False, min_value=1)
    overround = mongoengine.FloatField(required=False)

    meta = {
        "db_alias": "features",
        "collection": CONFIG["connections"]["features"]["odds"],
        "ordering": ["-received_at"],
    }

    @classmethod
    def get_latest(
        cls,
        ra_game_id: str,
        market: str,
        date: datetime.datetime = None,
    ):
        """
        Extract latest odds available for a given game prior to a given date
        
        Parameters
        ----------
        ra_game_id : str
            real-analytics game identifier.
        market: str
            type of market required; should one of MARKET_TYPES
        date: datetime
            find the lastest data document prior to `date`
        
        """
        if date is None:
             date = datetime.datetime.now()
        if market == "asian":
            # asian lines
            odds_feats = cls.objects(game_id=ra_game_id,
                                     market_type=market,
                                     received_at__lt=date).order_by("-received_at")
            # case empty
            if odds_feats is None:
                return None

            # get all lines
            all_lines = pd.DataFrame()
            for odds in odds_feats:
                odds_df = pd.DataFrame({x: odds[x] for x in odds}, index=[0])
                all_lines = pd.concat([all_lines,odds_df])
            
            # remove duplicates and keep the fresh one
            all_lines = all_lines.drop_duplicates(subset=['game_id', 'line_id'], keep= 'first')
            # calculate delta between odds1 and odds 2
            all_lines["delta"] = abs(all_lines["odds1"] - 2.0) + abs(all_lines["odds2"] - 2.0)  
            # keep the line that has a minimum delta (even line)
            line = all_lines.loc[all_lines['delta'] == all_lines['delta'].min()]
            
            return line
            
        else:
            return (
            cls.objects(game_id=ra_game_id, market_type=market, received_at__lt=date)
            .order_by("-received_at")
            .first()
        )

    @classmethod
    def get_odds_features(
        cls,
        ra_game_id: str,
        market: str,
        date: datetime.datetime = None,
    ) -> pd.DataFrame:
        """
        Extract odds features and compute implied probs
        
        Parameters
        ----------
        ra_game_id : str
            real-analytics game identifier.
        market: str
            type of market required; should one of MARKET_TYPES
        date: datetime
            find the lastest data document prior to `date`
            
        Returns
        -------
        1-row Data.Frame with odds columns and probabilities columns
        """
        if date is None:
            date = datetime.datetime.now()
        # extract squad info
        odds_feats = cls.get_latest(ra_game_id, market, date)
        if odds_feats is None:
            return None

        # convert to dataFrame
        odds_df = pd.DataFrame({x: odds_feats[x] for x in odds_feats}, index=[0])

        ## compute probs
        odds_cols = odds_df.columns[odds_df.columns.str.startswith("odds")].tolist()
        probs_cols = [re.sub("odds", "proba", col) for col in odds_cols]
        odds_values = odds_df.loc[:, odds_cols]
        probs = pd.DataFrame(
            (1 / odds_values.values) / (1 / odds_values.values).sum(),
            index=[0],
            columns=probs_cols,
        )

        return pd.concat([odds_df, probs], axis=1)

# 1X2 Example

In [ ]:
# | include: false

# initialise connections
mongo_init("prod_atlas")

ra_game_id = "97139def0420937a29c18cfbd7cbef970997d1b944d8e6c673cb0e65d178fb6e"
market = "1x2"

odds_feats = MarketOdds.get_odds_features(ra_game_id, market)
odds_feats

,id,game_id,game_opta_id,game_date,competition_id,competition_opta_id,season_id,home_team_id,away_team_id,bookmaker_id,...,market_type,line_id,received_at,odds1,odds2,oddsX,overround,proba1,proba2,probaX
0,97139def0420937a29c18cfbd7cbef970997d1b944d8e6...,97139def0420937a29c18cfbd7cbef970997d1b944d8e6...,None,2021-09-10 19:00:00,8b7f4583d7b21ff32b5b6046c2580bf2c7ca796d46451b...,None,2021,d20ced25be4443e85b4fac7736f7de23bbf24488948987...,a5f1ba9bce1e0710f24fffb2a76df274fc0288a014d165...,18,...,1x2,None,2021-09-10 18:59:22,3.68,2.23,3.3,None,0.265578,0.438263,0.296159


# Asian Handicap Example

In [ ]:
# | include: false

ra_game_id = "97139def0420937a29c18cfbd7cbef970997d1b944d8e6c673cb0e65d178fb6e"
market = "asian"

odds_feats = MarketOdds.get_latest(ra_game_id, market)
odds_feats

,id,game_id,game_opta_id,game_date,competition_id,competition_opta_id,season_id,home_team_id,away_team_id,bookmaker_id,bookmaker_name,source,market_type,line_id,received_at,odds1,odds2,oddsX,overround,delta
0,97139def0420937a29c18cfbd7cbef970997d1b944d8e6...,97139def0420937a29c18cfbd7cbef970997d1b944d8e6...,None,2021-09-10 19:00:00,8b7f4583d7b21ff32b5b6046c2580bf2c7ca796d46451b...,None,2021,d20ced25be4443e85b4fac7736f7de23bbf24488948987...,a5f1ba9bce1e0710f24fffb2a76df274fc0288a014d165...,18,PinnacleSports,oddsportal,asian,0.25,2021-09-10 18:32:09,2.08,1.85,None,NaN,0.23
